In [65]:
from sympy import *
x,y = symbols('x y')
f = Function('f')(x)

# two linear operators
#
# A1 corresponds to   Af = xf'
# A2 corresponds to   Af = (x+1)f'
def A1(f):
    return x * f.diff(x,1)

def A2(f):
    return (x+1) * f.diff(x,1)

# C computes the iterated function composition
#
# ( A o A o ... o A o f)
#   \............../
#    n times
#
def C(A,f,n):
    res = f
    for i in range(n):
        res = expand(A(res))
    return res

# convert f(x) (d/dx)^k -> y^k
#
# this function just searches for all derivatives f^(k)(x) and replaces them with y**k.
# the resulting expression is easier to handle in subsequent transformations.
def T(A):
    res = A
    if isinstance(A,Add):
        for u in A.args:
            u1 = u.args
            for i1 in range(len(u1)):
                if isinstance(u1[i1],Derivative):
                    u2 = u1[i1].args
                    if u2[0]==f:
                        k = u2[1][1]
                        res = res.replace(f.diff(x,k),y**k)
    elif isinstance(A,Mul):
        res = T(A+1)-1
    return collect(res,y)

c1 = C(A2,f,10)
display(T(c1))


y**10*(x**10 + 10*x**9 + 45*x**8 + 120*x**7 + 210*x**6 + 252*x**5 + 210*x**4 + 120*x**3 + 45*x**2 + 10*x + 1) + y**9*(45*x**9 + 405*x**8 + 1620*x**7 + 3780*x**6 + 5670*x**5 + 5670*x**4 + 3780*x**3 + 1620*x**2 + 405*x + 45) + y**8*(750*x**8 + 6000*x**7 + 21000*x**6 + 42000*x**5 + 52500*x**4 + 42000*x**3 + 21000*x**2 + 6000*x + 750) + y**7*(5880*x**7 + 41160*x**6 + 123480*x**5 + 205800*x**4 + 205800*x**3 + 123480*x**2 + 41160*x + 5880) + y**6*(22827*x**6 + 136962*x**5 + 342405*x**4 + 456540*x**3 + 342405*x**2 + 136962*x + 22827) + y**5*(42525*x**5 + 212625*x**4 + 425250*x**3 + 425250*x**2 + 212625*x + 42525) + y**4*(34105*x**4 + 136420*x**3 + 204630*x**2 + 136420*x + 34105) + y**3*(9330*x**3 + 27990*x**2 + 27990*x + 9330) + y**2*(511*x**2 + 1022*x + 511) + y*(x + 1)